## Data pre-processing

In [ ]:
import tokenizers
import torch
from tqdm.auto import tqdm
from datasets import load_dataset
from torch.utils.data import DataLoader

In [2]:
dataset = load_dataset("dair-ai/emotion")

In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})

In [4]:
dataset['train'][:5]

{'text': ['i didnt feel humiliated',
  'i can go from feeling so hopeless to so damned hopeful just from being around someone who cares and is awake',
  'im grabbing a minute to post i feel greedy wrong',
  'i am ever feeling nostalgic about the fireplace i will know that it is still on the property',
  'i am feeling grouchy'],
 'label': [0, 0, 3, 2, 3]}

In [5]:
import tiktoken

In [6]:
token_encoder = tiktoken.get_encoding("gpt2")

In [7]:
token_encoder.n_vocab

50257

In [8]:
def encode_text(x):
    text = str(x['text'])
    num_classes = 5
    max_seq_len = 36
    
    output = token_encoder.encode(text)
    output = output if len(output) <= max_seq_len else output[:max_seq_len]
    
    padding_length = max_seq_len - len(output)
    if padding_length > 0:
        output += [0] * padding_length
    
    label = [0 for _ in range(num_classes)]
    label[x['label']-1] = 1
    
    result = {
        'text': text,
        'encoded_text': output,
        'label': label
    }
    return result


In [9]:
tokenized_dataset_train = dataset['train'].map(encode_text)
tokenized_dataset_test = dataset['test'].map(encode_text)
tokenized_dataset_validation = dataset['validation'].map(encode_text)

In [10]:
len(tokenized_dataset_train[8]['encoded_text'])

36

In [11]:
train_dataloader = DataLoader(tokenized_dataset_train, batch_size=128, shuffle=True)
test_dataloader = DataLoader(tokenized_dataset_test, shuffle=True)
val_dataloader = DataLoader(tokenized_dataset_validation, batch_size=128, shuffle=True)

## Model Building

In [15]:
import sys
sys.path.append('..')

In [16]:
from model.transformers import EncoderClassifier
import torch

In [17]:
num_classes = 5
max_seq_len = 36

In [18]:
config = {
    "num_layers": 4,
    "vocab_size": token_encoder.n_vocab,
    "embed_dims": 768,
    "max_seq_len": max_seq_len,
    "n_segments": 5,
    "heads": 8,
    "dropout": 0.3,
    "device": "cpu",
    "ff_layer_sizes": [768, 256, 768],
    "batch_size": 128,
    "num_classes": 5
}

In [19]:
model = EncoderClassifier(config)

In [20]:
from tqdm.autonotebook import tqdm
from torch.optim import Adam

optim = Adam(model.parameters(), lr=1e-4)
criterion = torch.nn.CrossEntropyLoss()

In [21]:
epochs = 100
step = 0
device = config['device']

for epoch in range(epochs):
    loop = tqdm(train_dataloader, leave=True)
    
    model.train()
    
    total_loss = 0.0
    correct_predictions = 0
    for batch in loop:
        optim.zero_grad()

        inputs = torch.stack(batch['encoded_text']).int()
        labels = torch.stack(batch['label']).float()
        labels = torch.Tensor(labels).to(device)
        labels = labels.transpose(0, 1)
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        loss.backward()
        optim.step()

        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())
        step += 1
        
        total_loss += loss.item()
        _, labels = torch.max(labels, 1)
        _, predicted = torch.max(outputs, 1)
        correct_predictions += (predicted == labels).sum().item()

    average_loss = total_loss / len(train_dataloader)
    accuracy = correct_predictions / 16000
    
    print(f"Epoch {epoch + 1}/{epochs}, Loss: {average_loss:.4f}, Accuracy: {accuracy:.4f}")
    torch.save(model.state_dict(), f'models/model_epoch_{epoch + 1}.pth')

  0%|          | 0/125 [00:00<?, ?it/s]

Epoch 0: 100%|██████████| 125/125 [08:34<00:00,  4.12s/it, loss=1.52]


Epoch 1/100, Loss: 1.5163, Accuracy: 0.3371


Epoch 1: 100%|██████████| 125/125 [08:21<00:00,  4.01s/it, loss=1.51]


Epoch 2/100, Loss: 1.4928, Accuracy: 0.3862


...

Epoch 61/100, Loss: 0.9413, Accuracy: 0.9666


Epoch 61: 100%|██████████| 125/125 [08:04<00:00,  3.87s/it, loss=0.942]


Epoch 62/100, Loss: 0.9403, Accuracy: 0.9669


Epoch 62:  51%|█████     | 64/125 [04:11<03:59,  3.93s/it, loss=0.941]


KeyboardInterrupt: 